In [ ]:
import pandas as pd
import plotly.express as px
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import timedelta
import plotly.graph_objects as go

### Cargando Data

In [25]:
df_sales = pd.read_csv('data/daily_sales.csv')
df_factors = pd.read_csv('data/external_factors.csv')
df_hierarchy = pd.read_csv('data/product_hierarchy.csv')
df_promotion = pd.read_csv('data/promotional_calendar.csv')
df_inventary = pd.read_csv('data/store_inventory.csv')

df_sales['fecha'] = pd.to_datetime(df_sales['fecha'], format='%Y-%m-%d')
df_factors['fecha'] = pd.to_datetime(df_factors['fecha'], format='%Y-%m-%d')
df_promotion['fecha_inicio'] = pd.to_datetime(df_promotion['fecha_inicio'], format='%Y-%m-%d')
df_promotion['fecha_fin'] = pd.to_datetime(df_promotion['fecha_fin'], format='%Y-%m-%d')
df_inventary['fecha_inicio'] = pd.to_datetime(df_inventary['fecha_inicio'], format='%Y-%m-%d')


### Creando Features

In [32]:
df_sales_factor = pd.merge(df_sales, df_factors, on=['region', 'fecha'], how='inner')

# Agregar columnas derivadas de la fecha
df_sales_factor['año'] = df_sales_factor['fecha'].dt.year
df_sales_factor['mes'] = df_sales_factor['fecha'].dt.month
df_sales_factor['dia'] = df_sales_factor['fecha'].dt.day
df_sales_factor['dia_semana'] = df_sales_factor['fecha'].dt.weekday  # 0 = lunes
df_sales_factor['semana'] = df_sales_factor['fecha'].dt.isocalendar().week
# df_sales_factor['dia_agno_pasado'] = df_sales_factor['fecha'] - timedelta(days=364)

### Haciendo modelos

In [ ]:
resultados = {}
df_predicciones = pd.DataFrame()
products_list=list(df_sales['sku'].unique())

for sku in products_list:
    print(f"Entrenando modelo para {sku}...")
    
    df_sku = df_sales_factor[df_sales_factor['sku'] == sku].copy()
    fechas = df_sku['fecha']
    unidades = df_sku['unidades']

    # Variables predictoras (features)
    features = ['region', 'tienda', 'familia', 'precio_unitario',
                'temperatura', 'indice_economico', 'año', 'mes', 'dia', 'dia_semana', 'semana']
    
    # One-hot encoding para variables categóricas
    df_model = pd.get_dummies(df_sku[features + ['unidades']], drop_first=True)

    X = df_model.drop('unidades', axis=1)
    y = df_model['unidades']

    # Split train/test
    original_indices = df_sku.index
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, original_indices, test_size=0.2, random_state=42
)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Modelo
    model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # Evaluación
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    resultados[sku] = rmse

    print(f"RMSE para {sku}: {rmse:.2f}")
    df_preds = pd.DataFrame({
        'fecha': df_sku.loc[idx_test, 'fecha'].values,
        'real': y_test.values,
        'pred': y_pred,
        'SKU': sku
    })

    df_predicciones = pd.concat([df_predicciones, df_preds], ignore_index=True)

Entrenando modelo para SKU_01...
RMSE para SKU_01: 121.79
Entrenando modelo para SKU_02...
RMSE para SKU_02: 94.54
Entrenando modelo para SKU_03...
RMSE para SKU_03: 61.01
Entrenando modelo para SKU_04...
RMSE para SKU_04: 63.96
Entrenando modelo para SKU_05...
RMSE para SKU_05: 59.16
Entrenando modelo para SKU_06...
RMSE para SKU_06: 28.93
Entrenando modelo para SKU_07...
RMSE para SKU_07: 53.58
Entrenando modelo para SKU_08...
RMSE para SKU_08: 38.77
Entrenando modelo para SKU_09...
RMSE para SKU_09: 30.97
Entrenando modelo para SKU_10...
RMSE para SKU_10: 34.94
Entrenando modelo para SKU_11...
RMSE para SKU_11: 43.38
Entrenando modelo para SKU_12...
RMSE para SKU_12: 25.50


### Resultado grafico por producto

In [ ]:

sku_a_graficar = 'SKU_02'
df_plot = df_predicciones[df_predicciones['SKU'] == sku_a_graficar].sort_values('fecha')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_plot['fecha'], y=df_plot['real'],
    mode='lines+markers', name='Real'
))

fig.add_trace(go.Scatter(
    x=df_plot['fecha'], y=df_plot['pred'],
    mode='lines+markers', name='Predicho'
))

fig.update_layout(
    title=f'Predicción vs Real para {sku_a_graficar}',
    xaxis_title='Fecha',
    yaxis_title='Unidades',
    height=500
)

fig.show()


### Creado data para hacer predicciones a 7 días

In [ ]:
columnas = ['fecha', 'region', 'tienda', 'sku', 'familia', 'unidades', 'precio_unitario', 
            'temperatura', 'indice_economico', 'año', 'mes', 'dia', 
            'dia_semana', 'semana']
df_sales_factor = pd.DataFrame(columns=columnas)
fechas_pred = pd.date_range(start='2025-02-01', end='2025-02-07')

In [ ]:
fechas_pred = pd.date_range(start='2025-02-01', end='2025-02-07')

# Lista de tiendas (supongamos que es una lista conocida)
tiendas = list(df_sales_factor['tienda'].unique())

# Crear todas las combinaciones posibles: tienda x producto x fecha
combinaciones = [(fecha, tienda, sku) for fecha in fechas_pred for tienda in tiendas for sku in products_list]

df_pred_base = pd.DataFrame(combinaciones, columns=['fecha', 'tienda', 'sku'])

df_tiendas_regiones = df_sales[['tienda', 'region']].drop_duplicates()

In [ ]:
df_pred_base = df_pred_base.merge(df_tiendas_regiones, on='tienda', how='left')

In [62]:
df_ultimos_precios = df_sales[df_sales['fecha'] == '2024-12-30'][['tienda', 'sku', 'precio_unitario', 'familia']]
df_ultimos_precios = df_ultimos_precios.drop_duplicates(subset=['tienda', 'sku'])
df_pred_base = df_pred_base.merge(df_ultimos_precios, on=['tienda', 'sku'], how='left')


In [ ]:
temperaturas_region_dia = {
    '2025-02-01': {'Norte': 30.2, 'Centro': 26.3, 'Sur': 22.5},
    '2025-02-02': {'Norte': 30.0, 'Centro': 26.0, 'Sur': 22.0},
    '2025-02-03': {'Norte': 29.8, 'Centro': 25.7, 'Sur': 21.7},
    '2025-02-04': {'Norte': 30.5, 'Centro': 26.5, 'Sur': 22.3},
    '2025-02-05': {'Norte': 31.0, 'Centro': 27.0, 'Sur': 22.8},
    '2025-02-06': {'Norte': 30.3, 'Centro': 26.2, 'Sur': 22.1},
    '2025-02-07': {'Norte': 29.7, 'Centro': 25.8, 'Sur': 21.9}
}
df_pred_base['temperatura'] = df_pred_base.apply(
    lambda row: temperaturas_region_dia[row['fecha'].strftime('%Y-%m-%d')][row['region']],
    axis=1
)
df_pred_base

In [68]:
df_pred_base['indice_economico']=98.1
df_pred_base['año'] = df_pred_base['fecha'].dt.year
df_pred_base['mes'] = df_pred_base['fecha'].dt.month
df_pred_base['dia'] = df_pred_base['fecha'].dt.day
df_pred_base['dia_semana'] = df_pred_base['fecha'].dt.weekday
df_pred_base['semana'] = df_pred_base['fecha'].dt.isocalendar().week


In [ ]:
# Diccionario para almacenar resultados
df_predicciones_final = pd.DataFrame()

for sku in products_list:
    print(f"Prediciendo para {sku}...")
    
    # Filtrar por SKU
    df_pred_sku = df_pred_base[df_pred_base['sku'] == sku].copy()

    # Features que usaste para entrenamiento
    features = ['region', 'tienda', 'familia', 'precio_unitario',
                'temperatura', 'indice_economico', 'año', 'mes', 'dia', 'dia_semana', 'semana']

    df_pred_model = pd.get_dummies(df_pred_sku[features], drop_first=True)


    df_train_sku = df_sales_factor[df_sales_factor['sku'] == sku]
    df_train_model = pd.get_dummies(df_train_sku[features + ['unidades']], drop_first=True)
    X_train = df_train_model.drop('unidades', axis=1)
    y_train = df_train_model['unidades']

    model = XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    model.fit(X_train, y_train)

    # Predecir
    y_pred = model.predict(df_pred_model)

    # Guardar resultados
    df_result = df_pred_sku[['fecha', 'region', 'tienda', 'sku']].copy()
    df_result['unidades_pred'] = y_pred

    df_predicciones_final = pd.concat([df_predicciones_final, df_result], ignore_index=True)

df_predicciones_final['unidades_pred'] = df_predicciones_final['unidades_pred'].astype(int)
print("Predicción terminada.")


Prediciendo para SKU_01...
Prediciendo para SKU_02...
Prediciendo para SKU_03...
Prediciendo para SKU_04...
Prediciendo para SKU_05...
Prediciendo para SKU_06...
Prediciendo para SKU_07...
Prediciendo para SKU_08...
Prediciendo para SKU_09...
Prediciendo para SKU_10...
Prediciendo para SKU_11...
Prediciendo para SKU_12...
Predicción terminada.


In [ ]:
def plot_pred_vs_real(df, tienda_seleccionada):
    # Filtrar por tienda
    df_filtrado = df[df['tienda'] == tienda_seleccionada]

    fig = px.line(
        df_filtrado, 
        x='fecha', 
        y='unidades_pred', 
        color='sku', 
        title=f'Venta real vs predicción - Tienda {tienda_seleccionada}',

    )
    fig.show()

plot_pred_vs_real(df_predicciones_final, tienda_seleccionada='Tienda_02')

In [ ]:
df_predicciones_final.to_csv('data/forecast_csv')

,fecha,region,tienda,sku,unidades_pred
3,2025-02-01,Norte,Tienda_01,SKU_01,1169.224731
11,2025-02-02,Norte,Tienda_01,SKU_01,1162.581055
19,2025-02-03,Norte,Tienda_01,SKU_01,716.008423
27,2025-02-04,Norte,Tienda_01,SKU_01,721.853821
35,2025-02-05,Norte,Tienda_01,SKU_01,722.551514
...,...,...,...,...,...
635,2025-02-03,Norte,Tienda_01,SKU_12,37.976643
643,2025-02-04,Norte,Tienda_01,SKU_12,37.976643
651,2025-02-05,Norte,Tienda_01,SKU_12,37.731274
659,2025-02-06,Norte,Tienda_01,SKU_12,31.222298
